# Proyecto

# Libraries

In [1]:
import serial
import sys
import glob
import time
import numpy as np

# Environment

In [2]:
prod = True

# Functions

In [3]:
def listSerialPorts():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

def connectToPort(n):
    ports = listSerialPorts()
    return serial.Serial(ports[n-1])

def readValueFromData(data):
    # +000.000/0 G S
    dataString = data.decode("utf-8").replace('\r\n','')
    # print(data)
    if dataString.endswith(' G S'):
        result = dataString.replace(' G S','').replace('/','').replace('+','')
        return float(result)
    else:
        raise Exception("Invalied format")
    
def serialRead(connection,testValue=0,testMargin=0):
    if prod:
        while True:
            try:
                connection.flushInput()
                return readValueFromData(connection.readline())
            except:
                pass
    else:
        return testValue+np.random.uniform(-testMargin,testMargin)
    

# Connection

In [4]:

ports = listSerialPorts()

for i in range(0,len(ports)):
    print('{0}: {1}'.format(i+1, ports[i]))

1: /dev/tty.wlan-debug
2: /dev/tty.BT-163
3: /dev/tty.Bluetooth-Incoming-Port
4: /dev/tty.usbserial-FTDCYOPE


In [5]:
port = 4

connection = connectToPort(port)

In [6]:
# for i in range(0,10):
#     serialRead(connection)
#     time.sleep(1)

In [7]:
# connection.close()

# Procedure

Data

In [8]:
scaleDivision = 0.0001 # g
minimumCharge = 0.01 # g
nominalCalibrationMass = 200 # g
massCorrection = 0 # g
EMPs = { # EMP para masa E1 según la OIML R 111
    0.01: 0.00002,
    20: 0.000025,
    40: 0.00005,
    60: 0.00005,
    80: 0.000075,
    100: 0.00005,
    120: 0.000075,
    140: 0.0001,
    160: 0.0001,
    180: 0.00125,
    200: 0.0001,
    220: 0.000125
}


In [9]:
# import random

In [10]:
print('Iniciando procedimiento de calibración de la balanza.')
input('Presione enter para continuar.')

T0 = -200
while T0 == -200:
      try:
            T0 = float(input('Favor ingrese la temperatura en °C en este momento.'))
      except:
            print('Número inválido, favor ingrese un número válido: número racional')
            T0 = -200
            
P0 = 0
while P0 == 0:
      try:
            P0 = float(input('Favor ingrese la presión en hPa en este momento.'))
      except:
            print('Número inválido, favor ingrese un número válido: número racional positivo')
            P0 = 0
            
HR0 = 0
while HR0 == 0:
      try:
            HR0 = float(input('Favor ingrese la humedad relativa en este momento.'))
      except:
            print('Número inválido, favor ingrese un número válido: número racional positivo menor a 100')
            HR0 = 0


print('\nOperation conditions (initial)')

print(T0)
print(P0)
print(HR0)

# PRUEBA 1

print('|'+'-'*120+'|')

print('''
Prueba 1: Prueba de excentricidad

Se debe tomar un patrón de peso de 100 g y utilizarse en todos los puntos de la prueba conforme se indique en pantalla.
El patrón no se debe cambiar bajo ninguna circunstancia.
      ''')

input('Presione enter para continuar.')

points = ['Central','Inferior izquierdo','Superior izquierdo','Superior derecho','Inferior derecho','Central']

eccentricityPoints = []
for i in range(0,3):
      eccentricityPoints0 = []
      for i in range(0,len(points)):
            retry = True
            currentPoint = 0
            while retry:
                  print('\nCalibración del punto {0}, favor posicionar el patrón en el punto {1}'.format(i+1,points[i]))
                  keep = True
                  
                  while keep:
                        value = serialRead(connection,100,2)
                        if value >= nominalCalibrationMass*0.40:
                              keep = False
                              currentPoint=value
                  print('\nFavor retire el partón de la balanza y espere a que llegue a 0 (máximo 30 s).')
                  keep = True
                  ti = time.time()
                  while keep:
                        value = serialRead(connection,0,0)
                        if value == 0:
                              keep = False
                              retry = False
                        elif time.time()-ti <= 20:
                              keep = True
                        else:
                              retry = True
                              print(retry)
                              input('No se logró llegar a 0 en este punto, favor retirar la pesa y tarar para repetir el punto.')
                              
            eccentricityPoints0.append(currentPoint)
      
      eccentricityPoints.append(eccentricityPoints0)

print(np.array(eccentricityPoints))
eccentricityPoints = np.transpose(np.mean(np.array(eccentricityPoints),axis=1))

p0 = eccentricityPoints[-1]

# print(eccentricityPoints)

# print((np.vectorize(lambda row: np.abs(row[0]-row[1]-p0)))(eccentricityPoints))

# print(np.apply_along_axis(lambda row: np.abs(row[0]-row[1]-p0), 1, eccentricityPoints))

dEccentricityMax = np.max(np.apply_along_axis(lambda row: np.abs(row-p0), 0, eccentricityPoints))

print('\nEccentricity points\n')

print(eccentricityPoints)

print(dEccentricityMax)

# PRUEBA 2

print('|'+'-'*120+'|')

print('''
Prueba 2: Prueba por error de indicación

Se deben tomar los patrones según se van indicando en pantalla e ir posicionándolos en la pesa.
Si no se cuenta con el valor indicado o éste no es un valor estandar para las pesas de calibración, utilizar varias,
de tal forma que sumen la cantidad necesaria de masa.
Es recomendado utilizar una sola pesa para cada valor (ej. siempre utilizar la misma pesa de 200 g cuendo se necesite).
      ''')

input('Presione enter para continuar.')

indicationPoints = []
EMPkeys = list(EMPs.keys())
for i in range(0,len(EMPkeys)):
      retry = True
      currentPoint = [EMPkeys[i]]
      while retry:
            print('\nPosicionar {0} g en la balanza.'.format(EMPkeys[i]))
            keep = True
            
            while keep:
                  value = serialRead(connection,EMPkeys[i],2)
                  if value >= EMPkeys[i]*0.98:
                        keep = False
                        currentPoint.append(value)
            
            print('\nFavor retire el partón de la balanza.')
            keep = True
            ti = time.time()
            while keep:
                  value = serialRead(connection,0,0)
                  if value == 0:
                        keep = False
                        retry = False
                        currentPoint.append(value)
                  elif time.time()-ti <= 20:
                        keep = True
                  else:
                        retry = True
                        input('No se logró llegar a 0 en este punto, favor retirar la pesa y tarar para repetir el punto.')
    
      indicationPoints.append(currentPoint)

indicationPoints = np.array(indicationPoints)

# Punto de calibración (g), Indicación de la balanza (g), Indicación de descarga (g), Error de indicación (g), Masa de referencia (g)
indicationPoints = np.concatenate((indicationPoints,
                                   np.transpose(np.array([np.apply_along_axis(lambda row: row[1]-row[0]-EMPs[row[0]], 1, indicationPoints)])),
                                   np.transpose(np.array([np.apply_along_axis(lambda row: row[0]+EMPs[row[0]], 1, indicationPoints)]))
                                   ), axis=1)

print('\nIndication Points\n')

print(indicationPoints)


# PRUEBA 3

print('|'+'-'*120+'|')

print('''
Prueba 3: Prueba de repetibilidad

Se deben tomar los patrones para el 50% de carga ({0:.2f} g) y para el 100% de carga ({1:.2f} g) y posicionarlos según se indique.
Es recomendado utilizar una sola pesa para cada valor (ej. siempre utilizar la misma pesa de 200 g cuendo se necesite).
      '''.format(nominalCalibrationMass/2,nominalCalibrationMass))

rep = 0
while rep == 0:
      try:
            rep = int(input('Favor ingrese el número de repeticiones que se desean .'))
      except:
            print('Número inválido, favor ingrese un número válido de repeticiones: entero positivo mayor a 0')
            rep = 0

repetic = np.arange(1,rep,1)

print(f"Iniciando prueba de repetibilidad para {rep} mediciones")
input("Presione enter para continuar")

print('''Paso 1: Carga y descarga al 50% ({0:.2f} g)'''.format(nominalCalibrationMass/2))

input("Presione enter para continuar")


carga_50 = nominalCalibrationMass/2 # carga al 50% 


# CARGA AL 50 %
repetib50 = []

for i in repetic:
      
      print(f"{i}: Coloque la carga para el 50%")
      keep = True
      ind50 = []
      retry = True
      currentPoint = [EMPkeys[i]]
      while retry:
            while keep:   # carga al 50
                  value = serialRead(connection,carga_50,2)
                  if value >= carga_50*0.9:
                        keep = False
                        ind50.append(value)
            print(f"\nFavor retirar la carga")

            keep = True
            while keep:
                  value = serialRead(connection,0,0)
                  if value == 0:
                        keep = False
                        retry = False
                        ind50.append(value)
                  elif time.time()-ti <= 20:
                        keep = True
                  else:
                        retry = True
                        input('No se logró llegar a 0 en este punto, favor retirar la pesa y tarar para repetir el punto.')
            
      repetib50.append(ind50)

repetib50 = np.array(repetib50)



# CARGA AL 100%
repetib100 = []

print("Paso 2: carga y descarga al 100% ({0:.2f} g)".format(nominalCalibrationMass))
input("Presione enter para continuar")


for i in repetic:
      retry = True
      currentPoint = [EMPkeys[i]]
      while retry:
            print(f"{i}: Coloque la carga para el 100%")
            keep = True
            ind100 = []
            while keep:   
                  value = serialRead(connection,nominalCalibrationMass,2)
                  if value >= nominalCalibrationMass*0.9:
                        keep = False
                        ind100.append(value)
            print(f"\nFavor retirar la carga")

            keep = True
            while keep:
                  value = serialRead(connection,0,1)
                  if value == 0:
                        keep = False
                        retry = False
                        ind100.append(value)
                  elif time.time()-ti <= 20:
                        keep = True
                  else:
                        retry = True
                        input('No se logró llegar a 0 en este punto, favor retirar la pesa y tarar para repetir el punto.')
      
      repetib100.append(ind100)

repetib100 = np.array(repetib100)

print(repetib50)
print(repetib100)

S_Carg50 = np.std(repetib50[:,0], ddof=1) # desviacion estandar muestral de la carga a 50%
S_desc50 = np.std(repetib50[:,1], ddof=1) # desviacion estandar muestral de la descarga a 50%
S_Carg100 = np.std(repetib100[:,0], ddof=1) # desviacion estandar muestral de la carga a 100%
S_desc100 = np.std(repetib100[:,1], ddof=1) # desviacion estandar muestral de la descarga a 100%

print(S_Carg50)
print(S_desc50)
print(S_Carg100)
print(S_desc100)

TF = -200
while TF == -200:
      try:
            TF = float(input('Favor ingrese la temperatura en °C en este momento.'))
      except:
            print('Número inválido, favor ingrese un número válido: número racional')
            TF = -200
            
PF = 0
while PF == 0:
      try:
            PF = float(input('Favor ingrese la presión en hPa en este momento.'))
      except:
            print('Número inválido, favor ingrese un número válido: número racional positivo')
            PF = 0
            
HRF = 0
while HRF == 0:
      try:
            HRF = float(input('Favor ingrese la humedad relativa en este momento.'))
      except:
            print('Número inválido, favor ingrese un número válido: número racional positivo menor a 100')
            HRF = 0

print('\nOperation conditions (final)')

print(TF)
print(PF)
print(HRF)

Iniciando procedimiento de calibración de la balanza.

Operation conditions (initial)
23.6
861.0
83.0
|------------------------------------------------------------------------------------------------------------------------|

Prueba 1: Prueba de excentricidad

Se debe tomar un patrón de peso de 100 g y utilizarse en todos los puntos de la prueba conforme se indique en pantalla.
El patrón no se debe cambiar bajo ninguna circunstancia.
      

Calibración del punto 1, favor posicionar el patrón en el punto Central

Favor retire el partón de la balanza y espere a que llegue a 0 (máximo 30 s).

Calibración del punto 2, favor posicionar el patrón en el punto Inferior izquierdo

Favor retire el partón de la balanza y espere a que llegue a 0 (máximo 30 s).

Calibración del punto 3, favor posicionar el patrón en el punto Superior izquierdo

Favor retire el partón de la balanza y espere a que llegue a 0 (máximo 30 s).

Calibración del punto 4, favor posicionar el patrón en el punto Superior der

In [ ]:
print(indicationPoints[:,4])

# print(np.array(list(EMPs.items()))[:,1])

# print(np.array(list(EMPs.items()))[:,1]/(4*indicationPoints[:,4]))

# print(2.00e-05/(4*2.00000020e+02))


[ 10.00002   20.000025  40.00005   60.00005   80.000075 100.00005
 120.000075 140.0001   160.0001   180.00125  200.0001   220.000125]


In [ ]:
# ures y ures0, son iguales entonces se suman en una sola
ures = (scaleDivision/np.sqrt(6))*np.ones((1,len(indicationPoints)))

# Resultado de los paso 1 y 2

uexc = np.apply_along_axis(lambda row: row[1]*dEccentricityMax/(2*np.sqrt(3)*(nominalCalibrationMass+massCorrection)), 1, indicationPoints)

# Resultado del paso 3 REVISAR
urep = np.max([S_Carg50,S_Carg100])*np.ones((1,len(indicationPoints)))/np.sqrt(rep)

# Incertidumbre de excentricidad
u2l = ures**2+uexc**2+urep**2

utrazc = np.array(list(EMPs.items()))[:,1]/np.sqrt(3/2)

uder = np.array(list(EMPs.items()))[:,1]/(3*np.sqrt(3))

# REVISAR
paire = (0.348444*P0-HR0*(0.00252*T0-0.020582))/(273.15+T0)
ppesa = 7950 #kg/m3
uempuje = (0.1*paire/8000)*indicationPoints[:,4]/np.sqrt(3) + np.array(list(EMPs.items()))[:,1]/(4*indicationPoints[:,4])# 0*np.ones((1,len(indicationPoints)))

# REVISAR
uconv = 0*np.ones((1,len(indicationPoints)))

# print(utrazc)

# Incertidumbre de masa de referencia
u2m = utrazc**2+uder**2+uempuje**2+uconv**2

# Incertidumbre combinada
ue = np.sqrt(u2m + u2l)

print(u2m)

veff = (u2m**4)/((urep**4)/(len(indicationPoints)-1)+(utrazc**4)/50)

print(veff)

# k = veff.apply()

# k=0
# a = (1-coverage)/2
# if vef >= 200:
#     if rect:
#         pass
#     elif trap:
#         pass   
#     else:
#         k = scipy.stats.norm.ppf(1-a)-scipy.stats.norm.ppf(0.5)
# else:
#     k = scipy.stats.t.ppf(1-a,int(vef.evalf()))


connection.close()


[[7.86836402e-09 3.05482406e-08 1.21976163e-07 2.71976259e-07
  4.84284048e-07 7.51976531e-07 1.08428447e-06 1.47747218e-06
  1.92747247e-06 3.53498646e-06 3.00747307e-06 3.64154078e-06]]
[[9.81699733e-31 2.23041517e-28 5.66944794e-26 1.40140893e-24
  1.40877473e-23 8.18951239e-23 3.54008625e-22 1.22044482e-21
  3.53503433e-21 3.99937305e-20 2.09530806e-20 4.50382606e-20]]
